<a href="https://colab.research.google.com/github/martinbedia/mineria/blob/main/ejercicio%201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicio 1
Sobre el código realizado durante las clases para analizar tickets estudia si los tickets
contienen parking o no. Aquellos que lo contienen aportan información adicional. Modifica el programa para tener esto en cuenta. El resultado del código deberán ser dos archivos csv. Uno tendrá las columnas id ticket, producto, cantidad, precio por unidad. El otro tendrá para cada id de ticket, parking (booleana), fecha, total precio ticket, hora entrada, hora salida, tiempo total (que podéis calcularlo de las anteriores). Las columnas para las que no se dispone de datos en algunos de los tickets deberán ser NA.

In [3]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.7 MB/s eta 0:00:00


In [10]:
import sklearn
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.cluster import KMeans, AgglomerativeClustering
import matplotlib.pyplot as plt
import seaborn as sns
import glob, os
import PyPDF2
import re
import csv

In [5]:
# leer archivos de un tipo concreto de una carpeta y sus subcarpetas
# recursive para que busque en las subcarpetas
ruta_carpeta_raiz = '.'
tickets = glob.glob(os.path.join(ruta_carpeta_raiz, '**/*.pdf'),recursive=True)
tickets

['./Ticket-23_11_2023.pdf',
 './Ticket-13_11_2023.pdf',
 './Ticket-3_11_2023.pdf',
 './20230909 Mercadona 21,23 €.pdf']

Veo que me salen los 3 tickets sobre los que trabajaremos.

In [6]:
# leer el ticket
pdf_reader = PyPDF2.PdfReader(tickets[3])    # conectar el archivo pdf con el codigo en python
ticket = pdf_reader.pages[0]   # acceder a la primera pagina
texto_ticket = ticket.extract_text()
texto_ticket

'MERCADONA, S.A.   A-46103834\nPLAZA DAOIZ Y VELARDE 5\n33009 OVIEDO\nTELÉFONO: 985216153\n09/09/2023 20:35  OP: 191484\nFACTURA SIMPLIFICADA: 3433-011-051093\nDescripción P. Unit Importe\n2TAQUITOS SALMÓN 3,80 7,60\n1TROTTOLE TRICOLORE 0,95\n1QUESO RALLADO GOUDA 1,44\n1ÑOQUIS DE PATATA 1,12\n2LECHE SEMI S/LACT 1,46 2,92\n1QUESO FETA 2,35\n1PICATOSTE AJO 0,85\n2NEGRA RODAJAS P3 2,00 4,00\n1PARKING 0,00\nENTRADA  20:22       SALIDA  20:35\nTOTAL (€) 21,23\nTARJETA BANCARIA 21,23\nIVA BASE IMPONIBLE (€) CUOTA (€)\n10% 11,32 1,13\n5% 1,97 0,10\n0% 6,71 0,00\nTOTAL 20,00 1,23\nTARJ. BANCARIA:  **** **** **** 4016\nN.C: 003943818  AUT: 729144\nAID: A0000000041010   ARC: 3030\nMASTERCARD\nImporte: 21,23 € MASTERCARD\nSE ADMITEN DEVOLUCIONES CON TICKET\nDISPONE DE 20 MINUTOS\nPARA RETIRAR SU VEHÍCULO'

In [7]:
# texto del ticket por lineas
lineas = np.array(texto_ticket.split('\n'))
lineas

array(['MERCADONA, S.A.   A-46103834', 'PLAZA DAOIZ Y VELARDE 5',
       '33009 OVIEDO', 'TELÉFONO: 985216153',
       '09/09/2023 20:35  OP: 191484',
       'FACTURA SIMPLIFICADA: 3433-011-051093',
       'Descripción P. Unit Importe', '2TAQUITOS SALMÓN 3,80 7,60',
       '1TROTTOLE TRICOLORE 0,95', '1QUESO RALLADO GOUDA 1,44',
       '1ÑOQUIS DE PATATA 1,12', '2LECHE SEMI S/LACT 1,46 2,92',
       '1QUESO FETA 2,35', '1PICATOSTE AJO 0,85',
       '2NEGRA RODAJAS P3 2,00 4,00', '1PARKING 0,00',
       'ENTRADA  20:22       SALIDA  20:35', 'TOTAL (€) 21,23',
       'TARJETA BANCARIA 21,23', 'IVA BASE IMPONIBLE (€) CUOTA (€)',
       '10% 11,32 1,13', '5% 1,97 0,10', '0% 6,71 0,00',
       'TOTAL 20,00 1,23', 'TARJ. BANCARIA:  **** **** **** 4016',
       'N.C: 003943818  AUT: 729144', 'AID: A0000000041010   ARC: 3030',
       'MASTERCARD', 'Importe: 21,23 € MASTERCARD',
       'SE ADMITEN DEVOLUCIONES CON TICKET', 'DISPONE DE 20 MINUTOS',
       'PARA RETIRAR SU VEHÍCULO'], dtype='<U37

Meto lo anterior en una funcion a la que le diga la ruta y me devuelva el array con las lineas



In [31]:
# lineas_ticekt1 = leer_lineas_de_pdf(archivo)

def leer_lineas_de_pdf (nombre_del_pdf):
  pdf_reader = PyPDF2.PdfReader(nombre_del_pdf)
  ticket = pdf_reader.pages[0]
  texto_ticket = ticket.extract_text()
  lineas = np.array(texto_ticket.split('\n'))
  return lineas

In [94]:
lineas_ticekt1 = leer_lineas_de_pdf('/content/20230909 Mercadona 21,23 €.pdf')
lineas_ticekt1

array(['MERCADONA, S.A.   A-46103834', 'PLAZA DAOIZ Y VELARDE 5',
       '33009 OVIEDO', 'TELÉFONO: 985216153',
       '09/09/2023 20:35  OP: 191484',
       'FACTURA SIMPLIFICADA: 3433-011-051093',
       'Descripción P. Unit Importe', '2TAQUITOS SALMÓN 3,80 7,60',
       '1TROTTOLE TRICOLORE 0,95', '1QUESO RALLADO GOUDA 1,44',
       '1ÑOQUIS DE PATATA 1,12', '2LECHE SEMI S/LACT 1,46 2,92',
       '1QUESO FETA 2,35', '1PICATOSTE AJO 0,85',
       '2NEGRA RODAJAS P3 2,00 4,00', '1PARKING 0,00',
       'ENTRADA  20:22       SALIDA  20:35', 'TOTAL (€) 21,23',
       'TARJETA BANCARIA 21,23', 'IVA BASE IMPONIBLE (€) CUOTA (€)',
       '10% 11,32 1,13', '5% 1,97 0,10', '0% 6,71 0,00',
       'TOTAL 20,00 1,23', 'TARJ. BANCARIA:  **** **** **** 4016',
       'N.C: 003943818  AUT: 729144', 'AID: A0000000041010   ARC: 3030',
       'MASTERCARD', 'Importe: 21,23 € MASTERCARD',
       'SE ADMITEN DEVOLUCIONES CON TICKET', 'DISPONE DE 20 MINUTOS',
       'PARA RETIRAR SU VEHÍCULO'], dtype='<U37

Ahora, una vez que tengo la informacion, debo extraer el id del ticket, los productos, la cantidad y el precio por unidad. Usare expresiones regulares para identificar esa informacion en el texto en bruto.

In [155]:
def procesar_ticket_lineas(lineas):
    id_ticket_match = re.search(r'\d*-\d*-\d*', '\n'.join(lineas))
    id_ticket = id_ticket_match.group() if id_ticket_match else None

    productos = []
    for linea in lineas:
        producto_match = re.search(r'\d[A-ZÑñÁÉÍÓÚáéíóú][[A-ZÑñÁÉÍÓÚáéíóú/0-9\s]*((\d{1,2},\d{1,2}\s*){1,2})', linea, flags=re.IGNORECASE)
        if producto_match:
            productos.append(producto_match.group())

    return id_ticket, productos

In [172]:
ticket1_prepro = procesar_ticket_lineas(lineas_ticekt1)
ticket1_pro = ticket1_prepro[1]
ticket1_pro

['2TAQUITOS SALMÓN 3,80 7,60',
 '1TROTTOLE TRICOLORE 0,95',
 '1QUESO RALLADO GOUDA 1,44',
 '1ÑOQUIS DE PATATA 1,12',
 '2LECHE SEMI S/LACT 1,46 2,92',
 '1QUESO FETA 2,35',
 '1PICATOSTE AJO 0,85',
 '2NEGRA RODAJAS P3 2,00 4,00',
 '1PARKING 0,00']

In [180]:
def procesar_productos(lineas):
    cantidad = re.search(r'^(\d{1,})', '\n'.join(lineas))



    return cantidad

In [182]:
def obtener_cantidad(linea):
    # Patrón para extraer el número inicial de la línea
    patron = r'^(\d{1,})'

    # Buscar coincidencias en la línea
    coincidencia = re.search(patron, linea)

    if coincidencia:
        return coincidencia.group()
    else:
        return None

# Obtener las cantidades
cantidades = [obtener_cantidad(linea) for linea in ticket1_pro]

# Imprimir las cantidades
print("Cantidades:", cantidades)


Cantidades: ['2', '1', '1', '1', '2', '1', '1', '2', '1']


In [189]:
def obtener_nombre_producto(linea):
    # Patrón para extraer el nombre del producto
    patron = r'^\d{1,}([A-ZÑñÁÉÍÓÚáéíóú/0-9\s]*)'

    # Buscar coincidencias en la línea
    coincidencia = re.search(patron, linea, flags=re.IGNORECASE)

    if coincidencia:
        return coincidencia.group(1).strip()
    else:
        return None

# Obtener los nombres de los productos
nombres_productos = [obtener_nombre_producto(linea) for linea in ticket1_pro]

# Imprimir los nombres de los productos
print("Nombres de Productos:", nombres_productos)

Nombres de Productos: ['TAQUITOS SALMÓN 3', 'TROTTOLE TRICOLORE 0', 'QUESO RALLADO GOUDA 1', 'ÑOQUIS DE PATATA 1', 'LECHE SEMI S/LACT 1', 'QUESO FETA 2', 'PICATOSTE AJO 0', 'NEGRA RODAJAS P3 2', 'PARKING 0']


In [192]:
df = pd.DataFrame({
    'id ticket': id_ticket,
    'cantidad': cantidades,
    'producto': nombres_productos,
    'precio_individual': cantidades
})
df

,id ticket,cantidad,producto,precio_individual
0,3433-011-051093,2,TAQUITOS SALMÓN 3,2
1,3433-011-051093,1,TROTTOLE TRICOLORE 0,1
2,3433-011-051093,1,QUESO RALLADO GOUDA 1,1
3,3433-011-051093,1,ÑOQUIS DE PATATA 1,1
4,3433-011-051093,2,LECHE SEMI S/LACT 1,2
5,3433-011-051093,1,QUESO FETA 2,1
6,3433-011-051093,1,PICATOSTE AJO 0,1
7,3433-011-051093,2,NEGRA RODAJAS P3 2,2
8,3433-011-051093,1,PARKING 0,1
